In [ ]:
!sudo apt update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,533 kB]
Hit:13 http://ppa.launchpad.net/cran

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz


In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession
         .builder
         .appName("Chess_Tournament_Analysis")
         .getOrCreate())


In [ ]:
import gdown

url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/chess/chess_schema.png',
 '/content/chess/chess_wc_history_game_info.csv',
 '/content/chess/chess_wc_history_moves.csv',
 '/content/chess/eco_codes.csv']

In [ ]:
df1 = spark.read.load("/content/chess/chess_wc_history_game_info.csv", format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
df2 = spark.read.load("/content/chess/chess_wc_history_moves.csv", format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
df3 = spark.read.load("/content/chess/eco_codes.csv", format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
from pyspark.sql import functions as F


In [ ]:
#This function will accept spark dataframes and will store in a List.
#And at the end this list of spark dataframes will be converted into Pandas 
#And then it will be saved in the specified drive location in csv format.


lst_df = []
def my_function(dfff):
  lst_df.append(dfff)

**SOLUTION** **1** : Winners of Each World champions Trophy

In [ ]:
result1_df = df1.filter(F.col('tournament_name').contains('WorldChamp')).\
             filter(F.col('winner_loser_elo_diff') > 0).\
             groupBy('winner', 'tournament_name').\
             agg(F.sum('winner_loser_elo_diff').alias('count_val'))


In [ ]:
df_result2 = result1_df.groupBy('tournament_name').\
             agg(F.max('count_val').alias('mx'))

In [ ]:
df_r1 = result1_df.alias('tab1').\
       join(df_result2.alias('tab2'),\
            ((F.col('tab1.tournament_name') == F.col('tab2.tournament_name')) & (F.col('tab1.count_val') == F.col('tab2.mx'))), 'inner')

In [ ]:
df_r1 = df_r1.select(F.col('tab1.winner'), F.col('tab1.tournament_name'))

In [ ]:
df_r1 = df_r1.sort('tournament_name')

In [ ]:
df_r1.show()
my_function(df_r1)

+--------------------+---------------+
|              winner|tournament_name|
+--------------------+---------------+
|Fischer, Robert J...| WorldChamp1972|
|     Karpov, Anatoly| WorldChamp1978|
|     Karpov, Anatoly| WorldChamp1981|
|      Kasparov, Gary| WorldChamp1984|
|     Karpov, Anatoly| WorldChamp1985|
|      Kasparov, Gary| WorldChamp1986|
|      Kasparov, Gary| WorldChamp1987|
|      Kasparov, Gary| WorldChamp1990|
|           Kramnik,V| WorldChamp2004|
|           Topalov,V| WorldChamp2006|
|             Anand,V| WorldChamp2007|
|             Anand,V| WorldChamp2008|
|           Topalov,V| WorldChamp2010|
|             Anand,V| WorldChamp2012|
|  Anand, Viswanathan| WorldChamp2012|
|     Carlsen, Magnus| WorldChamp2013|
|     Carlsen, Magnus| WorldChamp2014|
|     Carlsen, Magnus| WorldChamp2016|
|           Carlsen,M| WorldChamp2021|
+--------------------+---------------+



**SOLUTION** **2** :  List of Players with number of times they have won Tournament

In [ ]:
df_r2 = df1.filter(F.col('winner')!='draw').\
        groupBy(F.col('winner').alias('player_name')).\
        agg(F.count(F.col('winner')).alias('number_of_wins')).\
        sort(F.col('number_of_wins').desc())

In [ ]:
df_r2.show()
my_function(df_r2)

+--------------------+--------------+
|         player_name|number_of_wins|
+--------------------+--------------+
|     Lasker, Emanuel|            52|
|     Karpov, Anatoly|            47|
|  Botvinnik, Mikhail|            46|
|   Steinitz, William|            43|
| Alekhine, Alexander|            43|
|           Topalov,V|            32|
|      Kasparov, Gary|            31|
|             Anand,V|            30|
|    Smyslov, Vassily|            24|
|            Adams,Mi|            20|
|      Kasimdzhanov,R|            19|
|          Grischuk,A|            17|
|           Svidler,P|            16|
|      Adams, Michael|            16|
|            Shirov,A|            15|
|             Dreev,A|            15|
|            Bareev,E|            15|
|   Chigorin, Mikhail|            14|
|Khalifman, Alexander|            14|
|           Euwe, Max|            14|
+--------------------+--------------+
only showing top 20 rows



**SOLUTION** **3** : Most and Least Popular eco move in world championship history.

In [ ]:
df_r3 = df1.filter(F.col('eco') != "").\
        groupBy('eco').agg(F.count(F.col('eco')).alias('number_of_occurances')).\
        sort(F.col('number_of_occurances').desc(), F.col('eco').asc())

In [ ]:
eco1 = df_r3.collect()[0][0]
eco2 = df_r3.collect()[df_r3.count() - 1][0]

In [ ]:
df_r33 = df_r3.filter((F.col('eco') == eco1) | (F.col('eco') == eco2))

In [ ]:
df_r33 = df_r33.alias('tab1').\
         join(df3.alias('tab2'),\
              (F.col('tab1.eco') == F.col('tab2.eco')), "inner")

In [ ]:
df_r33 = df_r33.select(F.col("tab1.eco"), F.col('tab2.eco_name'), F.col('tab1.number_of_occurances'))

In [ ]:
df_r33.show()
my_function(df_r33)

+---+--------------------+--------------------+
|eco|            eco_name|number_of_occurances|
+---+--------------------+--------------------+
|C42|      Petrov Defense|                  64|
|E99|King's Indian, Or...|                   1|
+---+--------------------+--------------------+



**SOLUTION** **4** : Eco move with most winnings

In [ ]:
df_4 = df1.filter(F.col('winner') != 'draw').\
       groupBy(F.col('eco')).agg(F.count(F.col('eco')).alias('eco_count')).\
       orderBy(F.col('eco_count').desc()).limit(1)


In [ ]:
df_4 = df_4.alias('tab1').join(df3.alias('tab2'),\
                               F.col('tab1.eco') == F.col('tab2.eco'), 'inner').\
        select(F.col('tab1.eco'), F.col('tab2.eco_name'))

In [ ]:
df_4.show()
my_function(df_4)

+---+-----------------+
|eco|         eco_name|
+---+-----------------+
|B90|Sicilian, Najdorf|
+---+-----------------+



**SOLUTION** **5** :  Longest and shortest game ever played in a world championship in terms of move

In [ ]:
df_51 = df2.groupBy('game_id').\
        agg(F.count(F.col('game_id')).alias('number_of_moves'))

In [ ]:
df_51 = df_51.select(F.col('game_id'),\
                     F.floor(F.col('number_of_moves') / 2).alias('number_of_moves')).\
        orderBy(F.col('number_of_moves').desc())

In [ ]:
df_52 = df1.filter(F.col('tournament_name').contains('WorldChamp')).\
        select(F.col('game_id'), F.col('event'), F.col('tournament_name'))

In [ ]:
df_5 = df_52.alias('tab1').join(df_51.alias('tab2'),\
                                F.col('tab1.game_id') == F.col('tab2.game_id'), 'inner').\
        select(F.col('tab1.game_id'), F.col('tab1.event'), F.col('tab1.tournament_name'), F.col('tab2.number_of_moves')).\
        orderBy(F.col('number_of_moves').desc())

In [ ]:
coll= df_5.collect()
lstn=[]
lstn.append([coll[0][0], coll[0][1],coll[0][2], coll[0][3] ])
lstn.append([coll[df_5.count() - 1][0], coll[df_5.count() - 1][1],coll[df_5.count() - 1][2], coll[df_5.count() - 1][3]])
df_5 = spark.createDataFrame(lstn, ['game_id', 'event', 'tournament_name','number_of_moves'])

In [ ]:
df_5.show()
my_function(df_5)

+--------------------+--------------------+---------------+---------------+
|             game_id|               event|tournament_name|number_of_moves|
+--------------------+--------------------+---------------+---------------+
|58e83255-93bb-4d5...|            WCh 2021| WorldChamp2021|            135|
|1846cede-0037-4f0...|World Championshi...| WorldChamp1972|              0|
+--------------------+--------------------+---------------+---------------+



**SOLUTION** **6** : Shortest and Longest Draw game ever Played.

In [ ]:
df_r61 = df2.groupBy('game_id').\
         agg(F.count(F.col('game_id')).alias('number_of_moves'))

In [ ]:
df_r61 = df_r61.select(F.col('game_id'),\
                       F.floor(F.col('number_of_moves') / 2).alias('number_of_moves')).\
                       orderBy(F.col('number_of_moves').desc())

In [ ]:
df_r62 = df1.filter(F.col('winner') == 'draw').\
         select(F.col('game_id'), F.col('event'), F.col('tournament_name'))

In [ ]:
df_r6 = df_r62.alias('tab1').join(df_r61.alias('tab2'),\
                                  F.col('tab1.game_id') == F.col('tab2.game_id'), 'inner').\
        select(F.col('tab1.game_id'), F.col('tab1.event'), F.col('tab1.tournament_name'), F.col('tab2.number_of_moves')).\
        orderBy(F.col('number_of_moves').desc())

In [ ]:
coll= df_r6.collect()
l=[]
l.append([coll[0][0], coll[0][1],coll[0][2], coll[0][3] ])
l.append([coll[df_r6.count() - 1][0], coll[df_r6.count() - 1][1],coll[df_r6.count() - 1][2], coll[df_r6.count() - 1][3]])
#print(l)
df_r6 = spark.createDataFrame(l, ['game_id', 'event', 'tournament_name','number_of_moves'])

In [ ]:
df_r6.show()
my_function(df_r6)

+--------------------+-------------+---------------+---------------+
|             game_id|        event|tournament_name|number_of_moves|
+--------------------+-------------+---------------+---------------+
|88f34084-e4df-490...|  FIDE WCh KO|  FideChamp2002|            129|
|a27c2b95-fb62-4b6...|FIDE-Wch k.o.|  FideChamp1998|              6|
+--------------------+-------------+---------------+---------------+



**SOLUTION** **7** : Most and Least rated Player

In [ ]:
df_r71 = df1.filter(F.col('winner') != 'draw').\
            groupBy(F.col('winner').alias('player')).\
              agg(F.sum(F.col('winner_elo')).alias('elo'))

In [ ]:
df_r72 = df1.filter(F.col('loser') != 'draw').\
           groupBy(F.col('loser').alias('player')).\
             agg(F.sum(F.col('loser_elo')).alias('elo'))

In [ ]:
df_r73 = df_r71.unionAll(df_r72)
df_r74 = df_r73.groupBy('player').\
         agg(F.sum(F.col('elo')).alias('elo')).\
         orderBy(F.col('elo').desc())

In [ ]:
df_r74 = df_r74.na.drop(how="any")

In [ ]:
coll= df_r74.collect()
l=[]
l.append([coll[0][0], coll[0][1] ])
l.append([coll[df_r74.count() - 1][0], coll[df_r74.count() - 1][1]])
df_r75 = spark.createDataFrame(l, ['player_name', 'elo'])

In [ ]:
df_r75.show()
my_function(df_r75)

+---------------+------+
|    player_name|   elo|
+---------------+------+
|Karpov, Anatoly|226160|
|    Utnasunov,A|  2257|
+---------------+------+



**SOLUTION** **8** : 3rd Last Player with most Loss

In [ ]:
df_r81 = df1.filter(F.col('loser') != 'draw').\
            groupBy(F.col('loser').alias('player_name')).\
              agg(F.count(F.col('loser')).alias('count')).\
                orderBy(F.col('count').asc(),F.col('player_name').asc())

In [ ]:
name = df_r81.collect()[df_r81.count() - 3][0]
df_8 = df_r81.filter(F.col('player_name') == name).select('player_name')
df_8.show()
my_function(df_r81)

+---------------+
|    player_name|
+---------------+
|Karpov, Anatoly|
+---------------+



**SOLUTION** **9** :  How many times players with low rating won matches with their total win Count

In [ ]:
avg = df_r74.agg({'elo' : 'avg'}).collect()[0][0]
df_r92 = df_r74.filter(F.col('elo') < avg)

In [ ]:
df_r91 = df1.filter(F.col('winner') != 'draw').\
           groupBy(F.col('winner').alias('player_name')).\
             agg(F.count(F.col('winner')).alias('win_count'))

In [ ]:
df_r93 = df_r91.alias('tab1').join(df_r92.alias('tab2'),\
                                   F.col('tab1.player_name') == F.col('tab2.player'), 'inner').\
        select(F.col('tab1.player_name'), F.col('tab1.win_count'))

In [ ]:
df_r93.show()
my_function(df_r93)

+------------------+---------+
|       player_name|win_count|
+------------------+---------+
|   Fridman, Daniel|        2|
|   Alterman, Boris|        1|
|   Jussupow, Artur|        1|
|          Piket,Je|        3|
|       Onischuk,Al|        1|
|    Kacheishvili,G|        1|
|          Luther,T|        3|
|         Carlsen,M|        4|
|         Ivanov,AV|        1|
|Istratescu, Andrei|        2|
| Miladinovic, Igor|        2|
|     Kotsur, Pavel|        1|
|      Benjamin,Joe|        3|
|       Yu Shaoteng|        1|
|    Benjamin, Joel|        1|
|     Adianto, Utut|        2|
|      Glek, Igor V|        1|
|    Shipov, Sergei|        1|
|Ponomariov, Ruslan|        2|
|            Pina,S|        1|
+------------------+---------+
only showing top 20 rows



**SOLUTION** **12** :  All Perfect Tournament with Winner Name

In [ ]:
df_r121 = df1.filter(F.col('winner') != 'draw').\
            groupBy('tournament_name').\
            agg(F.count(F.col('tournament_name')).alias('count_event'))

In [ ]:
df_r122 = df1.filter(F.col('winner') != 'draw').\
            groupBy(F.col('tournament_name'),F.col('winner')).\
            agg(F.count(F.col('winner')).alias('count_wins'))

In [ ]:
df_r123 = df_r121.alias('tab1').join(df_r122.alias('tab2'),\
                                     F.col('tab1.tournament_name') == F.col('tab2.tournament_name'), 'inner').\
          select(F.col('tab1.tournament_name'), F.col('tab2.winner'), F.col('tab2.count_wins'), F.col('tab1.count_event'))

In [ ]:
df_r124 = df_r123.filter(F.col('count_wins') == F.col('count_event')).\
             select(F.col('winner').alias('winner_name'), F.col('tournament_name'))

In [ ]:
df_r124.show()
my_function(df_r124)

+--------------------+---------------+
|         winner_name|tournament_name|
+--------------------+---------------+
|Capablanca, Jose ...| WorldChamp1921|
|           Carlsen,M| WorldChamp2021|
|     Lasker, Emanuel| WorldChamp1907|
|     Carlsen, Magnus| WorldChamp2013|
|     Lasker, Emanuel|WorldChamp1910b|
|           Kramnik,V| WorldChamp2000|
+--------------------+---------------+



**SOLUTION** **13** : Player with highest winning ratio

In [ ]:
df_131 = df1.groupBy(F.col('white').alias('player')).\
            agg(F.count(F.col('white')).alias('count'))

In [ ]:
df_132 = df1.groupBy(F.col('black').alias('player')).\
            agg(F.count(F.col('black')).alias('count'))

In [ ]:
df_133 = df_131.union(df_132).groupBy('player').\
            agg(F.sum(F.col('count')).alias('number_of_matches'))

In [ ]:
df_134 = df1.filter(F.col('winner') != 'draw').\
            groupBy(F.col('winner').alias('player')).\
              agg(F.count(F.col('winner')).alias('count_of_wins'))

In [ ]:
df_135 = df_134.alias('tab1').join(df_133.alias('tab2'),\
                                   F.col('tab1.player') == F.col('tab2.player'), 'inner').\
          select(F.col('tab1.player'), F.col('tab1.count_of_wins'), F.col('tab2.number_of_matches'))

In [ ]:
df_136 = df_135.select(F.col('player'),\
                       (F.col('count_of_wins')/F.col('number_of_matches')).alias('ratio')).\
                       orderBy(F.col('ratio').desc()).limit(1).\
                       select(F.col('player'))

In [ ]:
df_136.show()
my_function(df_136)

+--------+
|  player|
+--------+
|Karlov,A|
+--------+



**SOLUTION** **14** : Player who had given checkmate with Pawn

In [ ]:
df_141 = df2.filter((F.col('is_check_mate')==1)&(F.col('piece').contains('P'))).\
            select(F.col('player').alias('player_name'))


In [ ]:
df_141.show()
my_function(df_141)

+--------------+
|   player_name|
+--------------+
|Andersson, Ulf|
+--------------+



In [ ]:
import os
dir = "DE_SOLUTION_Ketan_Nalwaya"
pdir = "drive/MyDrive/"
path = os.path.join(pdir,dir)
os.mkdir(path)

In [ ]:
dr = "results"
pdr = "drive/MyDrive/DE_SOLUTION_Ketan_Nalwaya/"
pth = os.path.join(pdr,dr)
os.mkdir(pth)

In [ ]:
#Converting Spark Dataframes into Pandas
b=[]
for i in range(len(lst_df)):
  b.append(lst_df[i].toPandas())


In [ ]:
#Converting Pandas to csv and saving in the drive.
ph = 'drive/MyDrive/'
fnames = ['df1','df2','df3','df4','df5','df6','df7','df8','df9','df12','df13','df14']
for j in range(len(b)):
  b[j].to_csv(f'drive/MyDrive/DE_SOLUTION_Ketan_Nalwaya/results/{fnames[j]}.csv',header=True)